In [0]:
from keras import layers
from keras import Input
from keras.models import Model

'''
サンプルデータの生成
'''
vocabulary_size = 50000
num_income_groups = 10

'''
モデルの構築
'''
posts_input = Input(shape=(None,), dtype="int32", name="posts")
# Embedding(入力データの最大インデックス+1(max_features), 分散ベクトルの次元数, 入力の系列長) -> (batch_size(none), 入力の系列長, 分散ベクトルの次元数)
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation="relu")
x = layers.Conv1D(256, 5, activation="relu")
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation="relu")
x = layers.Conv1D(256, 5, activation="relu")
x = layers.GlobalMaxPooling1D()
x = layers.Dense(128, activation="relu")(x)

# 出力層に名前がついていることに注意
age_prediction = layers.Dense(1, name="age")(x)
income_prediction = layers.Dense(num_income_groups, activation="softmax", name="income")(x)
gender_prediction = layers.Dense(1, activation="sigmoid", name="gender")(x)
model = Model(
    posts_input,
    [age_prediction, income_prediction, gender_prediction]
)
model.compile(
    optimizer="rmsprop",
    loss=["mse", "categorical_crossentropy", "binary_crossentropy"],
    loss_weights=[0.25, 1., 10.]
)
# # 出力層に名前をつけている場合
# model.compile(
#     optimizer="rmsprop",
#     loss={"age": "mse", "income": "categorical_crossentropy", "gender": "binary_crossentropy"}
#     loss_weights={"age": 0.25, "income": 1., "gender": 10.}
# )

'''
学習
'''
# age_targets, income_targets, gender_targetsはNumPy配列と仮定
model.fit(
    posts,
    [age_targets, income_targets, gender_targets],
    epochs=10,
    batch_size=64
)
# # 出力層に名前をつけている場合
# model.fit(
#     posts,
#     {"age": age_targets, "income": income_targets, "gender": gender_targets},
#     epochs=10,
#     batch_size=64
# )
